In [1]:
#!pip install psycopg2-binary

In [2]:
import os, boto3, requests, io, json
from io import StringIO
from datetime import datetime, timedelta, timezone
import pandas as pd
from psycopg2 import pool, sql

In [8]:
with open("credentials/aws_keys.json") as f:
    keys = json.load(f)

os.environ["AWS_ACCESS_KEY_ID"] = keys["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] = keys["AWS_SECRET_ACCESS_KEY"]
os.environ["AWS_DEFAULT_REGION"] = keys["AWS_DEFAULT_REGION"]


with open("credentials/bbdd.json") as f:
    bbdd = json.load(f)

os.environ["host"] = bbdd["host"]
os.environ["database"] = bbdd["database"]
os.environ["user"] = bbdd["user"]
os.environ["password"] = bbdd["password"]

In [9]:
s3 = boto3.client("s3")

bucket_name = "hab-ree-data-json"
file_key = "merged/ree-data_full.json"

response = s3.get_object(Bucket=bucket_name, Key=file_key)
data_json = json.loads(response["Body"].read())
print(f"Se cargaron {len(data_json)} registros")

Se cargaron 760090 registros


In [10]:
connection_pool = pool.SimpleConnectionPool(
    1, 10,
    host=bbdd["host"],
    database=bbdd["database"],
    user=bbdd["user"],
    password=bbdd["password"]
)

conn = connection_pool.getconn()

connection_pool.putconn(conn)


OperationalError: connection to server at "13.62.85.98", port 5432 failed: FATAL:  too many connections for database "ree"


In [ ]:
table_name = "demanda_electrica"

columns = data_json[0].keys()

columns_with_types = ", ".join(f"{col} TEXT" for col in columns)  
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_with_types});"

conn = connection_pool.getconn()
cursor = conn.cursor()

cursor.execute(create_table_query)
conn.commit()
cursor.close()
connection_pool.putconn(conn)


In [ ]:
conn = connection_pool.getconn()
cursor = conn.cursor()

for row in data_json:
    cols = row.keys()
    values = [str(row[col]) for col in cols] 
    insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
        sql.Identifier(table_name),
        sql.SQL(', ').join(map(sql.Identifier, cols)),
        sql.SQL(', ').join(sql.Placeholder() * len(cols))
    )
    cursor.execute(insert_query, values)

conn.commit()
cursor.close()
connection_pool.putconn(conn)


In [ ]:
#cerrar las conexiones
connection_pool.closeall()
